<a href="https://colab.research.google.com/github/sharsulkar/H1B_LCA_outcome_prediction/blob/main/prototyping/notebooks/02_sh_build_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [35]:
observations_df=pd.read_csv('https://raw.githubusercontent.com/sharsulkar/H1B_LCA_outcome_prediction/main/reports/preprocessing_steps_observations.csv',sep='$',index_col=0,error_bad_lines=False)

In [36]:
#only pull columns that are required
required_features=list(observations_df[observations_df.preprocess_comment.isin([np.NaN,'Feature engineering','Target feature'])].index)

In [38]:
LCA_df=pd.read_excel('https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q2.xlsx',usecols=required_features)

In [39]:
#Drop rows where CASE_STATUS not in ('Certified','Denied')
LCA_df.drop(index=LCA_df[~LCA_df.CASE_STATUS.isin(['Certified','Denied'])].index,inplace=True)
LCA_df.reset_index(inplace=True)

### Feature engineering

In [7]:
#feature engineering steps
def date_diff(date1,date2):
  #error handling - 
    #validate input datatype is datetime
    #handle empty inputs
  return date1-date2

#### Processing_Days and Validity_days

In [40]:
LCA_df['PROCESSING_DAYS']=date_diff(LCA_df.DECISION_DATE, LCA_df.RECEIVED_DATE)
LCA_df['VALIDITY_DAYS']=date_diff(LCA_df.END_DATE, LCA_df.BEGIN_DATE)

#### SOC_Codes

In [41]:
LCA_df['SOC_CD2']=LCA_df.SOC_CODE.str.split(pat='-',n=1,expand=True)[0]

In [42]:
LCA_df['SOC_CD4']=LCA_df.SOC_CODE.str.split(pat='-',n=1,expand=True)[1].str.split(pat='.',n=1,expand=True)[0]

In [43]:
LCA_df['SOC_CD_ONET']=LCA_df.SOC_CODE.str.split(pat='-',n=1,expand=True)[1].str.split(pat='.',n=1,expand=True)[1]

#### USA_YN

In [51]:
def is_USA(country):
  if country=='UNITED STATES OF AMERICA':
    USA_YN='Y' 
  else:
    USA_YN='N'
  return USA_YN

In [53]:
LCA_df['USA_YN']=LCA_df.EMPLOYER_COUNTRY.apply(is_USA)

#### Employer_Worksite_YN

In [89]:
LCA_df['EMPLOYER_WORKSITE_YN']='Y'
LCA_df.loc[LCA_df.EMPLOYER_POSTAL_CODE.ne(LCA_df.WORKSITE_POSTAL_CODE),'EMPLOYER_WORKSITE_YN']='N'

#### OES_YN

In [99]:
LCA_df['OES_YN']='Y'
LCA_df.iloc[LCA_df[~LCA_df.PW_OTHER_SOURCE.isna()].index,LCA_df.columns.get_loc('OES_YN')]='N'

#### SURVEY_YEAR

In [98]:
def extract_survey_year(OES_YN,PW_OES_YEAR,PW_OTHER_YEAR):
  if OES_YN=='Y':
    #extract year from first date of PW_OES_YEAR
  else:
    #return PW_OTHER_YEAR 

43

#### WAGE_ABOVE_PREVAILING_HR

In [ ]:
def wage_per_hour(WAGE,UNIT_OF_PAY):
  #if UNIT_OF_PAY=='YEAR' then wage_per_hour=WAGE/2067
  #if UNIT_OF_PAY=='MONTH' then wage_per_hourWAGE/172
  #if UNIT_OF_PAY=='HOUR' then wage_per_hour=WAGE
  #return wage_per_hour

#### Drop columns after feature engineering

In [19]:
LCA_df.drop(columns=['DECISION_DATE','RECEIVED_DATE','END_DATE','BEGIN_DATE','SOC_CODE','EMPLOYER_POSTAL_CODE','WORKSITE_POSTAL_CODE','PW_OTHER_SOURCE','PW_OES_YEAR','PW_OTHER_YEAR'],inplace=True)

In [72]:
LCA_df.head()

,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,BEGIN_DATE,END_DATE,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,EMPLOYER_NAME,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,NAICS_CODE,AGENT_REPRESENTING_EMPLOYER,WORKSITE_WORKERS,SECONDARY_ENTITY,WORKSITE_POSTAL_CODE,WAGE_RATE_OF_PAY_FROM,PREVAILING_WAGE,PW_WAGE_LEVEL,PW_OES_YEAR,PW_OTHER_SOURCE,PW_OTHER_YEAR,TOTAL_WORKSITE_LOCATIONS,AGREE_TO_LC_STATEMENT,H-1B_DEPENDENT,WILLFUL_VIOLATOR,PUBLIC_DISCLOSURE,PROCESSING_DAYS,VALIDITY_DAYS,SOC_CD2,SOC_CD4,SOC_CD_ONET,USA_YN,EMPLOYER_WORKSITE_YN
0,Certified,2019-12-23,2020-01-01,H-1B,15-1121.00,Computer Systems Analysts,Y,2020-01-01,2022-12-31,1,0,1,0,0,0,0,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,77845,UNITED STATES OF AMERICA,541512,N,1.0,Y,7310,92435.00,92435.00,II,7/1/2019 - 6/30/2020,NaN,NaN,1.0,Y,Y,N,Disclose Business,9 days,1095 days,15,1121,00,Y,N
1,Certified,2019-12-23,2020-01-01,H-1B,17-2072.00,"Electronics Engineers, Except Computer",Y,2020-06-16,2023-06-15,30,5,5,5,5,5,5,"SYNOPSYS, INC.",94043,UNITED STATES OF AMERICA,541512,Y,30.0,N,94043,113684.00,113684.00,NaN,NaN,Survey,2019.0,1.0,Y,N,N,Disclose Business,9 days,1094 days,17,2072,00,Y,Y
2,Certified,2019-12-23,2020-01-01,H-1B,15-1121.00,Computer Systems Analysts,Y,2020-01-01,2022-12-31,1,0,1,0,0,0,0,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,77845,UNITED STATES OF AMERICA,541512,N,1.0,Y,30342,77771.00,77771.00,II,7/1/2019 - 6/30/2020,NaN,NaN,1.0,Y,Y,N,Disclose Business,9 days,1095 days,15,1121,00,Y,N
3,Certified,2019-12-23,2020-01-01,H-1B,15-1121.00,Computer Systems Analysts,Y,2020-01-01,2022-12-31,1,0,1,0,0,0,0,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,77845,UNITED STATES OF AMERICA,541512,N,1.0,Y,68106,65624.00,65624.00,II,7/1/2019 - 6/30/2020,NaN,NaN,1.0,Y,Y,N,Disclose Business,9 days,1095 days,15,1121,00,Y,N
4,Certified,2019-12-23,2020-01-01,H-1B,15-1132.00,"Software Developers, Applications",Y,2020-01-10,2023-01-09,1,0,0,0,0,1,0,Populus Group LLC,48084,UNITED STATES OF AMERICA,561320,Y,1.0,Y,94089,86.54,74.36,IV,7/1/2019 - 6/30/2020,NaN,NaN,1.0,Y,N,N,Disclose Business,9 days,1095 days,15,1132,00,Y,N


In [ ]:
#separate target column and convert to binary

In [ ]:
#embed categorical features

In [ ]:
#scale numerical features